# Exploration of skip connection implementation

## Import required Python modules

In [1]:
import sys
import os
import numpy as np
import pandas as pd
# mcfly
from mcfly import modelgen, find_architecture, storage
from keras.models import load_model
np.random.seed(2)

Using TensorFlow backend.


In [2]:
sys.path.insert(0, os.path.abspath('../..'))
from utils import tutorial_pamap2

## Download data pre-procesed data

We have created a function for you to fetch the preprocessed data from https://zenodo.org/record/834467. Please specify the `directory_to_extract_to` in the code below and then execute the cell. This will download the preprocessed data into the directory in the `data` subdirectory. The output of the function is the path where the preprocessed data was stored.

In [3]:
# Specify in which directory you want to store the data:
directory_to_extract_to = '.'

In [4]:
data_path = tutorial_pamap2.download_preprocessed_data(directory_to_extract_to)

Extracting data...
Done


In [5]:
data_path

'./data/PAMAP2/preprocessed'

## Load the pre-processed data

Load the preprocessed data as stored in Numpy-files. Please note that the data has already been split up in a training (training), validation (val), and test subsets. It is common practice to call the input data X and the labels y.

In [6]:
X_train, y_train_binary, X_val, y_val_binary, X_test, y_test_binary, labels = tutorial_pamap2.load_data(data_path)

Data X and labels y are of type Numpy array. In the cell below we inspect the shape of the data. As you can see the shape of X is expressed as a Python tuple containing: the number of samples, length of the time series, and the number of channels for each sample. Similarly, the shape of y is represents the number of samples and the number of classes (unique labels). Note that y has the format of a binary array where only the correct class for each sample is assigned a 1. This is called one-hot-encoding.

In [7]:
print('x shape:', X_train.shape)
print('y shape:', y_train_binary.shape)

x shape: (11397, 512, 9)
y shape: (11397, 7)


The data is split between train test and validation.

In [8]:
print('train set size:', X_train.shape[0])
print('validation set size:', X_val.shape[0])
print('test set size:', X_test.shape[0])

train set size: 11397
validation set size: 100
test set size: 1000


Let's have a look at the distribution of the labels:

In [9]:
frequencies = y_train_binary.mean(axis=0)
frequencies_df = pd.DataFrame(frequencies, index=labels, columns=['frequency'])
frequencies_df

,frequency
lying,0.136615
sitting,0.130736
standing,0.136703
walking,0.176625
cycling,0.118540
vaccuum_cleaning,0.125208
ironing,0.175573


### *Question 1: How many channels does this dataset have?*
### *Question 2: What is the least common activity label in this dataset?*

    

## Generate models

First step in the development of any deep learning model is to create a model architecture. As we do not know what architecture is best for our data we will create a set of random models to investigate which architecture is most suitable for our data and classification task. This process, creating random models, checking how good they are and then selecting the best one is called a 'random search'. A random search is considered to be the most robust approach to finding a good model. You will need to specificy how many models you want to create with argument 'number_of_models'. See for a full overview of the optional arguments the function documentation of modelgen.generate_models by running `modelgen.generate_models?`.

##### What number of models to select?
This number differs per dataset. More models will give better results but it will take longer to evaluate them. For the purpose of this tutorial we recommend trying only 2 models to begin with. If you have enough time you can try a larger number of models, e.g. 10 or 20 models. Because mcfly uses random search, you will get better results when using more models.

In [10]:
num_classes = y_train_binary.shape[1]

models = modelgen.generate_models(X_train.shape,
                                  number_of_classes=num_classes,
                                  number_of_models = 2)

In [11]:
X_train.shape

(11397, 512, 9)

In [12]:
from keras.layers import Input
from keras.layers.convolutional import Conv2D
from keras.layers import BatchNormalization, Activation, Convolution1D, Lambda, \
    Convolution2D, Flatten, \
    Reshape, LSTM, Dropout, TimeDistributed, BatchNormalization
from keras.regularizers import l2
import keras

In [13]:
x = Input(shape=(512, 3))

# 1x3 conv with 3 output channels (same as input channels)
y = Convolution1D(3, (3), padding='same')(x)
# this returns x + y.
z = keras.layers.add([x, y],name='skipconnection')

In [14]:
 nn = keras.models.Model(inputs=x, outputs=z)
        

In [15]:
nn.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 3)       0                                            
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 512, 3)       30          input_1[0][0]                    
__________________________________________________________________________________________________
skipconnection (Add)            (None, 512, 3)       0           input_1[0][0]                    
                                                                 conv1d_4[0][0]                   
Total params: 30
Trainable params: 30
Non-trainable params: 0
__________________________________________________________________________________________________
